In [ ]:
# single connect

In [4]:
pip install pandas scikit-learn

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 10.8 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 37.6/37.6 MB 11.1 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [8]:
import socket
import json
import time

# Server configuration
SERVER_IP = '192.168.0.116'    # or your server's IP address
SERVER_PORT = 14000
SHARED_PASSWORD = "my_secure_password"
END_OF_MSG = "<???DONE???---"

def send_message(sock, message):
    """Sends a message with the custom end-of-message marker."""
    full_message = message + END_OF_MSG
    sock.sendall(full_message.encode('utf-8'))

def receive_message(sock):
    """Receive data until the custom marker is found."""
    buffer = ""
    while END_OF_MSG not in buffer:
        data = sock.recv(1024).decode('utf-8')
        if not data:
            break
        buffer += data
    # Remove the marker and return clean message.
    return buffer.replace(END_OF_MSG, "").strip()


In [9]:
# Create a TCP connection to the server and send the password
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((SERVER_IP, SERVER_PORT))
    print("Connected to server.")

    # Send the shared password
    send_message(sock, SHARED_PASSWORD)
    # Wait for server response (e.g., "auth_success" or "auth_fail")
    response = receive_message(sock)
    print("Authentication response:", response)

    if response != "auth_success":
        raise Exception("Authentication failed. Check your password.")
except Exception as e:
    print("Error connecting or authenticating:", e)
    sock.close()


Connected to server.
Authentication response: auth_success


In [10]:
# Send a JSON command to request planet data
get_planets_cmd = {
    "type": "get_planets"
}
send_message(sock, json.dumps(get_planets_cmd))

# Wait and print the planet data response
planets_response = receive_message(sock)
print("Received planet data:")
print(planets_response)

# Example: Parse the JSON response (adjust parsing based on your JSON structure)
try:
    planets_data = json.loads(planets_response)
    # Assuming planets_data is a list of planets with positions; here we pick the first one.
    if isinstance(planets_data, list) and planets_data:
        selected_planet = planets_data[0]
        # Assume each planet has a "position" field with [x, y, z]
        planet_position = selected_planet.get("position", [0,0,0])
        print("Selected planet position:", planet_position)
    else:
        print("Planet data not in expected format.")
except Exception as e:
    print("Error parsing planet data:", e)


Received planet data:
{"(0, 0, 0)":[{"Position":{"x":0,"y":0,"z":0},"Seed":-1137706823,"Name":"(0, 0, 0)","ResourceLocations":[{"x":0.3737519,"y":0.21537592,"z":0.25282946},{"x":-0.038451567,"y":0.49459165,"z":0.062454604},{"x":0.0019477914,"y":0.47938702,"z":-0.14207149},{"x":0.3355466,"y":-0.27974346,"z":0.24321201},{"x":-0.032191407,"y":0.49875697,"z":-0.014324633},{"x":-0.06452342,"y":0.4557601,"z":-0.19524206},{"x":0.038518414,"y":-0.49849397,"z":-0.0044826823},{"x":-0.17041288,"y":0.027668262,"z":0.46924824},{"x":-0.0006309848,"y":0.49978003,"z":0.014816351},{"x":0.33135423,"y":0.34553376,"z":0.14425954},{"x":-0.01970671,"y":-0.4993764,"z":-0.015325747},{"x":-0.32521045,"y":0.37766024,"z":0.040135846},{"x":-0.11430149,"y":0.4730961,"z":-0.114521876},{"x":-0.13837458,"y":-0.47978696,"z":-0.025630958},{"x":-0.0877007,"y":-0.2532181,"z":0.42212456},{"x":0.3025402,"y":0.20570368,"z":-0.3408158},{"x":-0.16849813,"y":-0.11748678,"z":0.45585662},{"x":-0.01155089,"y":-0.49938002,"z":-0.0

In [11]:
import pandas as pd
from IPython.display import display

# Create a list to hold rows of data for the table.
table_data = []

# Iterate over each key-value pair in the planets_data dictionary.
# Each key is a coordinate (like "(0, 0, 0)") and the value is a list of planet dictionaries.
for coord, planets in planets_data.items():
    for planet in planets:
        # Extract position from the 'Position' dictionary.
        pos = planet.get("Position", {})
        x = pos.get("x", None)
        y = pos.get("y", None)
        z = pos.get("z", None)
        
        # Extract other details.
        name = planet.get("Name", "Unknown")
        seed = planet.get("Seed", None)
        biome = planet.get("BiomeType", None)
        
        # Append a row with the coordinate (as key) and extracted data.
        table_data.append({
            "Coordinate": coord,
            "Name": name,
            "X": x,
            "Y": y,
            "Z": z,
            "Seed": seed,
            "BiomeType": biome
        })

# Create a DataFrame and display it.
df = pd.DataFrame(table_data)
display(df)


,Coordinate,Name,X,Y,Z,Seed,BiomeType
0,"(0, 0, 0)","(0, 0, 0)",0,0,0,-1137706823,0
1,"(0, 0, 1)","(0, 0, 1)",0,0,800,-116099305,1
2,"(0, 0, 2)","(0, 0, 2)",0,0,1600,-1768693292,1
3,"(0, 1, 0)","(0, 1, 0)",0,800,0,2006072299,2
4,"(0, 1, 1)","(0, 1, 1)",0,800,800,-474678776,1
5,"(0, 1, 2)","(0, 1, 2)",0,800,1600,1219022241,2
6,"(0, 2, 0)","(0, 2, 0)",0,1600,0,149058420,1
7,"(0, 2, 1)","(0, 2, 1)",0,1600,800,-1141743011,2
8,"(0, 2, 2)","(0, 2, 2)",0,1600,1600,-242691379,1
9,"(1, 0, 0)","(1, 0, 0)",800,0,0,-1404967705,1


In [13]:
import socket
import json
import time

# Server configuration
SERVER_IP = '192.168.0.116'  # Replace with your server's IP
SERVER_PORT = 14000
SHARED_PASSWORD = "my_secure_password"
END_OF_MSG = "<???DONE???---"

def send_message(sock, message):
    """Sends a message with the custom end-of-message marker."""
    full_message = message + END_OF_MSG
    sock.sendall(full_message.encode('utf-8'))

def receive_message(sock):
    """Receive data until the custom marker is found."""
    buffer = ""
    while END_OF_MSG not in buffer:
        data = sock.recv(1024).decode('utf-8')
        if not data:
            break
        buffer += data
    return buffer.replace(END_OF_MSG, "").strip()

# Connect to the server and get planet data
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((SERVER_IP, SERVER_PORT))
    print("Connected to server.")

    # Authenticate
    send_message(sock, SHARED_PASSWORD)
    response = receive_message(sock)
    print("Authentication response:", response)
    if response != "auth_success":
        raise Exception("Authentication failed. Check your password.")

    # Request planet data
    get_planets_cmd = {"type": "get_planets"}
    send_message(sock, json.dumps(get_planets_cmd))
    planets_response = receive_message(sock)
    print("Received planet data:", planets_response)

    # Parse the JSON response
    planets_data = json.loads(planets_response)

    # Extract resources for planet (0, 0, 0) as an example
    planet_key = "(0, 0, 0)"
    if planet_key in planets_data:
        resources = planets_data[planet_key][0]["ResourceLocations"]
        print(f"\nResources on planet {planet_key}:")
        for i, resource in enumerate(resources[:10], 1):  # First 10
            print(f"{i}. x: {resource['x']}, y: {resource['y']}, z: {resource['z']}")
    else:
        print(f"Planet {planet_key} not found.")

except Exception as e:
    print("Error:", e)
finally:
    sock.close()
    print("Disconnected from server.")

Connected to server.
Authentication response: auth_success
Received planet data: {"(0, 0, 0)":[{"Position":{"x":0,"y":0,"z":0},"Seed":-1137706823,"Name":"(0, 0, 0)","ResourceLocations":[{"x":0.3737519,"y":0.21537592,"z":0.25282946},{"x":-0.038451567,"y":0.49459165,"z":0.062454604},{"x":0.0019477914,"y":0.47938702,"z":-0.14207149},{"x":0.3355466,"y":-0.27974346,"z":0.24321201},{"x":-0.032191407,"y":0.49875697,"z":-0.014324633},{"x":-0.06452342,"y":0.4557601,"z":-0.19524206},{"x":0.038518414,"y":-0.49849397,"z":-0.0044826823},{"x":-0.17041288,"y":0.027668262,"z":0.46924824},{"x":-0.0006309848,"y":0.49978003,"z":0.014816351},{"x":0.33135423,"y":0.34553376,"z":0.14425954},{"x":-0.01970671,"y":-0.4993764,"z":-0.015325747},{"x":-0.32521045,"y":0.37766024,"z":0.040135846},{"x":-0.11430149,"y":0.4730961,"z":-0.114521876},{"x":-0.13837458,"y":-0.47978696,"z":-0.025630958},{"x":-0.0877007,"y":-0.2532181,"z":0.42212456},{"x":0.3025402,"y":0.20570368,"z":-0.3408158},{"x":-0.16849813,"y":-0.1174867

In [18]:
import socket
import json
import time
import pandas as pd
from IPython.display import display  # Only works in Jupyter; comment out if not using Jupyter

# Server configuration
SERVER_IP = '192.168.0.116'  # Replace with your server's IP
SERVER_PORT = 14000
SHARED_PASSWORD = "my_secure_password"
END_OF_MSG = "<???DONE???---"

def send_message(sock, message):
    """Sends a message with the custom end-of-message marker."""
    full_message = message + END_OF_MSG
    sock.sendall(full_message.encode('utf-8'))

def receive_message(sock):
    """Receive data until the custom marker is found."""
    buffer = ""
    while END_OF_MSG not in buffer:
        data = sock.recv(1024).decode('utf-8')
        if not data:
            break
        buffer += data
    return buffer.replace(END_OF_MSG, "").strip()

# Connect to the server and get planet data
try:
    sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
    sock.connect((SERVER_IP, SERVER_PORT))
    print("Connected to server.")

    # Authenticate
    send_message(sock, SHARED_PASSWORD)
    response = receive_message(sock)
    print("Authentication response:", response)
    if response != "auth_success":
        raise Exception("Authentication failed. Check your password.")

    # Request planet data
    get_planets_cmd = {"type": "get_planets"}
    send_message(sock, json.dumps(get_planets_cmd))
    planets_response = receive_message(sock)
    # print("Received planet data:", planets_response)

    # Parse the JSON response
    planets_data = json.loads(planets_response)

    # Extract data for planet (0, 0, 0)
    planet_key = "(0, 0, 0)"
    if planet_key in planets_data:
        # Extract resources
        resources = planets_data[planet_key][0]["ResourceLocations"]
        print(f"\nProcessing resources on planet {planet_key} into a DataFrame...")
        resource_data = [
            {"Index": i, "x": resource["x"], "y": resource["y"], "z": resource["z"]}
            for i, resource in enumerate(resources[:10], 1)  # First 10 resources
        ]
        df_resources = pd.DataFrame(resource_data)
        df_resources.set_index("Index", inplace=True)

        # Extract trees
        trees = planets_data[planet_key][0]["TreeLocations"]
        print(f"\nProcessing trees on planet {planet_key} into a DataFrame...")
        tree_data = [
            {"Index": i, "x": tree["x"], "y": tree["y"], "z": tree["z"]}
            for i, tree in enumerate(trees[:10], 1)  # First 10 trees
        ]
        df_trees = pd.DataFrame(tree_data)
        df_trees.set_index("Index", inplace=True)

        # Display both DataFrames
        print("\nResources DataFrame:")
        try:
            display(df_resources)  # For Jupyter
        except NameError:
            print(df_resources)    # Fallback for standard Python

        print("\nTrees DataFrame:")
        try:
            display(df_trees)      # For Jupyter
        except NameError:
            print(df_trees)        # Fallback for standard Python

    else:
        print(f"Planet {planet_key} not found.")

except Exception as e:
    print("Error:", e)
finally:
    sock.close()
    print("Disconnected from server.")

Connected to server.
Authentication response: auth_success

Processing resources on planet (0, 0, 0) into a DataFrame...

Processing trees on planet (0, 0, 0) into a DataFrame...

Resources DataFrame:


,x,y,z
Index,,,
1,0.373752,0.215376,0.252829
2,-0.038452,0.494592,0.062455
3,0.001948,0.479387,-0.142071
4,0.335547,-0.279743,0.243212
5,-0.032191,0.498757,-0.014325
6,-0.064523,0.455760,-0.195242
7,0.038518,-0.498494,-0.004483
8,-0.170413,0.027668,0.469248
9,-0.000631,0.499780,0.014816



Trees DataFrame:


,x,y,z
Index,,,
1,-0.351909,-0.306069,0.180226
2,-0.411519,-0.279769,-0.048801
3,-0.003400,0.451381,0.215043
4,0.462104,-0.157183,0.108415
5,-0.276659,-0.414955,-0.035662
6,0.075175,0.491786,0.049954
7,0.300381,0.386075,-0.103525
8,-0.052651,-0.367118,0.335339
9,0.459851,-0.098406,0.169863


Disconnected from server.


In [29]:
import socket
import time
import json
import threading

# Configuration (adjust these if needed)
SERVER_IP = "192.168.0.116"          # Your server's IP
SERVER_PORT = 14000                  # Your server's TCP port
SHARED_PASSWORD = "my_secure_password"
END_OF_MSG = "<???DONE???---"

# Utility functions
def send_message(sock, message):
    full_message = message + END_OF_MSG
    sock.sendall(full_message.encode('utf-8'))

def receive_message(sock):
    buffer = ""
    try:
        while END_OF_MSG not in buffer:
            data = sock.recv(1024).decode('utf-8')
            if not data:
                break
            buffer += data
    except socket.timeout:
        pass  # Timeout reached; return what we have
    return buffer.replace(END_OF_MSG, "").strip()

def tcp_worker():
    try:
        # Create a new TCP connection and set a timeout
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1.0)  # 1-second timeout for recv
        sock.connect((SERVER_IP, SERVER_PORT))
        print("[Worker] Connected to server.")

        # Authenticate with the server
        send_message(sock, SHARED_PASSWORD)
        auth_response = receive_message(sock)
        print("[Worker] Authentication response:", auth_response)
        if auth_response != "auth_success":
            sock.close()
            print("[Worker] Authentication failed.")
            return

        # Define the planet's position and compute the spawn position (cube appears 5 units above the planet)
        #planet_position = [0, 0 + 100, 40]
        planet_position = [0, 0 + 100, 40]
        print("[Worker] Manually set planet_position to:", planet_position)
        base_spawn = planet_position[:]  # Copy the list
        base_spawn[1] += 5.0             # Offset in Y so the cube appears above the planet
        spawn_position = base_spawn      # Use this as the spawn position

        # Build and send the spawn_cube command
        spawn_cube_cmd = {
            "type": "spawn_cube",
            "position": spawn_position,
            "rotation": [0, 0, 0]
        }
        print("[Worker] Sending spawn_cube command:", spawn_cube_cmd)
        send_message(sock, json.dumps(spawn_cube_cmd))
        time.sleep(2)
        spawn_resp = receive_message(sock)
        print("[Worker] Spawn cube response (if any):", spawn_resp)

        # Build and send the apply_force command
        # This command includes a "force" field; no rotation is applied.
        force_vector = [0, 10, 4]  # Adjust this force as needed
        apply_force_cmd = {
            "type": "apply_force",
            "force": force_vector
        }
        print("[Worker] Sending apply_force command:", apply_force_cmd)
        send_message(sock, json.dumps(apply_force_cmd))
        time.sleep(2)
        force_resp = receive_message(sock)
        print("[Worker] Apply force response (if any):", force_resp)

        # Close the TCP connection
        sock.close()
        print("[Worker] Connection closed.")
    except Exception as e:
        print("[Worker] Exception occurred:", e)

# Start the TCP operations in a separate thread.
worker_thread = threading.Thread(target=tcp_worker)
worker_thread.start()
worker_thread.join()
print("TCP worker thread has completed.")


[Worker] Connected to server.
[Worker] Authentication response: auth_success
[Worker] Manually set planet_position to: [0, 100, 40]
[Worker] Sending spawn_cube command: {'type': 'spawn_cube', 'position': [0, 105.0, 40], 'rotation': [0, 0, 0]}
[Worker] Spawn cube response (if any): 
[Worker] Sending apply_force command: {'type': 'apply_force', 'force': [0, 10, 4]}
[Worker] Apply force response (if any): 
[Worker] Connection closed.
TCP worker thread has completed.


In [ ]:
# spawning inside the resource

In [33]:
import socket
import json
import time
import threading
import pandas as pd
import math

# Server configuration
SERVER_IP = '192.168.0.116'  # Replace with your server's IP
SERVER_PORT = 14000
SHARED_PASSWORD = "my_secure_password"
END_OF_MSG = "<???DONE???---"

def send_message(sock, message):
    """Sends a message with the custom end-of-message marker."""
    full_message = message + END_OF_MSG
    sock.sendall(full_message.encode('utf-8'))

def receive_message(sock):
    """Receive data until the custom marker is found."""
    buffer = ""
    try:
        while END_OF_MSG not in buffer:
            data = sock.recv(1024).decode('utf-8')
            if not data:
                break
            buffer += data
    except socket.timeout:
        pass  # Timeout reached; return what we have
    return buffer.replace(END_OF_MSG, "").strip()

def tcp_worker():
    try:
        # Create a new TCP connection and set a timeout
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.settimeout(1.0)  # 1-second timeout for recv
        sock.connect((SERVER_IP, SERVER_PORT))
        print("[Worker] Connected to server.")

        # Authenticate with the server
        send_message(sock, SHARED_PASSWORD)
        auth_response = receive_message(sock)
        print("[Worker] Authentication response:", auth_response)
        if auth_response != "auth_success":
            sock.close()
            print("[Worker] Authentication failed.")
            return

        # Request planet data
        get_planets_cmd = {"type": "get_planets"}
        send_message(sock, json.dumps(get_planets_cmd))
        planets_response = receive_message(sock)
        print("[Worker] Received planet data.")

        # Parse the JSON response
        planets_data = json.loads(planets_response)

        # Extract resources for planet (0, 0, 0)
        planet_key = "(0, 0, 0)"
        if planet_key in planets_data:
            resources = planets_data[planet_key][0]["ResourceLocations"]
            planet_position = planets_data[planet_key][0]["Position"]  # Planet's center
            print(f"[Worker] Planet {planet_key} center position: {planet_position}")
            print(f"[Worker] Processing resources on planet {planet_key} into a DataFrame...")

            # Create a DataFrame for the first 10 resources
            resource_data = [
                {"Index": i, "x": resource["x"], "y": resource["y"], "z": resource["z"]}
                for i, resource in enumerate(resources[:10], 1)
            ]
            df_resources = pd.DataFrame(resource_data)
            df_resources.set_index("Index", inplace=True)
            print("\nResources DataFrame (normalized):")
            print(df_resources)

            # Get the 10th resource's coordinates
            tenth_resource = df_resources.iloc[9]  # 10th row (1-based index in display)
            x, y, z = tenth_resource["x"], tenth_resource["y"], tenth_resource["z"]

            # Calculate the magnitude (distance from center) of the resource vector
            magnitude = math.sqrt(x**2 + y**2 + z**2)
            if magnitude == 0:
                print("[Worker] Warning: 10th resource at origin, using default direction.")
                x, y, z = 0, 1, 0  # Default to up direction if magnitude is zero
                magnitude = 1.0

            # Normalize the resource vector to unit length
            unit_x = x / magnitude
            unit_y = y / magnitude
            unit_z = z / magnitude

            # Planet radius and buffer
            planet_radius = 100.0
            buffer = 5.0  # Spawn 5 units above the surface

            # Scale to the surface (radius) and add buffer
            scaled_x = planet_position["x"] + (unit_x * (planet_radius + buffer))
            scaled_y = planet_position["y"] + (unit_y * (planet_radius + buffer))
            scaled_z = planet_position["z"] + (unit_z * (planet_radius + buffer))

            spawn_position = [scaled_x, scaled_y, scaled_z]
            print(f"[Worker] Spawn position (on surface with buffer) for 10th resource: {spawn_position}")

        else:
            print(f"[Worker] Planet {planet_key} not found.")
            sock.close()
            return

        # Build and send the spawn_cube command
        spawn_cube_cmd = {
            "type": "spawn_cube",
            "position": spawn_position,
            "rotation": [0, 0, 0]
        }
        print("[Worker] Sending spawn_cube command:", spawn_cube_cmd)
        send_message(sock, json.dumps(spawn_cube_cmd))
        time.sleep(2)
        spawn_resp = receive_message(sock)
        print("[Worker] Spawn cube response (if any):", spawn_resp)

        # Build and send the apply_force command
        force_vector = [0, 10, 4]  # Adjust this force as needed
        apply_force_cmd = {
            "type": "apply_force",
            "force": force_vector
        }
        print("[Worker] Sending apply_force command:", apply_force_cmd)
        send_message(sock, json.dumps(apply_force_cmd))
        time.sleep(2)
        force_resp = receive_message(sock)
        print("[Worker] Apply force response (if any):", force_resp)

        # Close the TCP connection
        sock.close()
        print("[Worker] Connection closed.")
    except Exception as e:
        print("[Worker] Exception occurred:", e)

# Start the TCP operations in a separate thread
worker_thread = threading.Thread(target=tcp_worker)
worker_thread.start()
worker_thread.join()
print("TCP worker thread has completed.")

[Worker] Connected to server.
[Worker] Authentication response: auth_success
[Worker] Received planet data.
[Worker] Planet (0, 0, 0) center position: {'x': 0, 'y': 0, 'z': 0}
[Worker] Processing resources on planet (0, 0, 0) into a DataFrame...

Resources DataFrame (normalized):
              x         y         z
Index                              
1      0.373752  0.215376  0.252829
2     -0.038452  0.494592  0.062455
3      0.001948  0.479387 -0.142071
4      0.335547 -0.279743  0.243212
5     -0.032191  0.498757 -0.014325
6     -0.064523  0.455760 -0.195242
7      0.038518 -0.498494 -0.004483
8     -0.170413  0.027668  0.469248
9     -0.000631  0.499780  0.014816
10     0.331354  0.345534  0.144260
[Worker] Spawn position (on surface with buffer) for 10th resource: [np.float64(69.58438552780794), np.float64(72.56208670917846), np.float64(30.29450219308874)]
[Worker] Sending spawn_cube command: {'type': 'spawn_cube', 'position': [np.float64(69.58438552780794), np.float64(72.5620867

In [1]:
import socket
import json
import time
import threading
import pandas as pd
import math
import os

# Server configuration
SERVER_IP = '192.168.0.116'  # Replace with your server's IP
SERVER_PORT = 14000
SHARED_PASSWORD = "my_secure_password"
END_OF_MSG = "<???DONE???---"

# Set a global default timeout for all sockets
socket.setdefaulttimeout(1.0)

def send_message(sock, message):
    """Sends a message with the custom end-of-message marker."""
    full_message = message + END_OF_MSG
    try:
        sock.sendall(full_message.encode('utf-8'))
    except (socket.error, OSError) as e:
        print(f"[Thread] Send error: {e}")

def receive_message(sock):
    """Receive data until the custom marker is found."""
    buffer = ""
    try:
        while END_OF_MSG not in buffer:
            data = sock.recv(1024).decode('utf-8')
            if not data:
                break
            buffer += data
    except (socket.timeout, socket.error, OSError):
        pass
    return buffer.replace(END_OF_MSG, "").strip()

def interpolate_positions(start_pos, end_pos, num_cubes, buffer=5.0):
    """Generate a list of positions between start_pos and end_pos with a buffer."""
    positions = []
    for i in range(num_cubes):
        t = i / (num_cubes - 1) if num_cubes > 1 else 0
        x = start_pos[0] + t * (end_pos[0] - start_pos[0])
        y = start_pos[1] + t * (end_pos[1] - start_pos[1])
        z = start_pos[2] + t * (end_pos[2] - start_pos[2])
        distance_from_center = math.sqrt(x**2 + y**2 + z**2)
        if distance_from_center < 100 + buffer:
            scale = (100 + buffer) / distance_from_center
            x *= scale
            y *= scale
            z *= scale
        positions.append([x, y, z])
    return positions

def spawn_cube_thread(cube_id, position, sockets_list):
    """Spawn a cube in a separate thread with its own socket."""
    sock = None
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((SERVER_IP, SERVER_PORT))
        print(f"[Cube {cube_id}] Connected to server.")

        # Authenticate
        send_message(sock, SHARED_PASSWORD)
        auth_response = receive_message(sock)
        print(f"[Cube {cube_id}] Authentication response:", auth_response)
        if auth_response != "auth_success":
            raise Exception(f"Cube {cube_id} authentication failed.")

        # Spawn the cube
        spawn_cube_cmd = {
            "type": "spawn_cube",
            "position": position,
            "rotation": [0, 0, 0]
        }
        print(f"[Cube {cube_id}] Sending spawn_cube command:", spawn_cube_cmd)
        send_message(sock, json.dumps(spawn_cube_cmd))
        # Not waiting for response to keep it fast
        print(f"[Cube {cube_id}] Spawn command sent.")

        # Add socket to list for later closure
        sockets_list.append(sock)
    except Exception as e:
        print(f"[Cube {cube_id}] Exception occurred:", e)
    # Note: We don’t close the socket here; main thread will handle it

def tcp_worker():
    sockets = []  # List to track all sockets
    threads = []  # List to track all threads

    try:
        # Initial connection to get planet data
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((SERVER_IP, SERVER_PORT))
        print("[Worker] Connected to server for planet data.")

        # Authenticate
        send_message(sock, SHARED_PASSWORD)
        auth_response = receive_message(sock)
        print("[Worker] Authentication response:", auth_response)
        if auth_response != "auth_success":
            sock.close()
            print("[Worker] Authentication failed.")
            return

        # Request planet data
        get_planets_cmd = {"type": "get_planets"}
        send_message(sock, json.dumps(get_planets_cmd))
        planets_response = receive_message(sock)
        print("[Worker] Received planet data.")

        # Parse the JSON response
        planets_data = json.loads(planets_response)
        sock.close()  # Close initial socket after getting data

        # Extract resources for planet (0, 0, 0)
        planet_key = "(0, 0, 0)"
        if planet_key in planets_data:
            resources = planets_data[planet_key][0]["ResourceLocations"]
            planet_position = planets_data[planet_key][0]["Position"]
            print(f"[Worker] Planet {planet_key} center position: {planet_position}")
            print(f"[Worker] Processing resources on planet {planet_key} into a DataFrame...")

            # Create a DataFrame for the first 10 resources
            resource_data = [
                {"Index": i, "x": resource["x"], "y": resource["y"], "z": resource["z"]}
                for i, resource in enumerate(resources[:10], 1)
            ]
            df_resources = pd.DataFrame(resource_data)
            df_resources.set_index("Index", inplace=True)
            print("\nResources DataFrame (normalized):")
            print(df_resources)

            # Get the 10th resource's coordinates
            tenth_resource = df_resources.iloc[9]
            x, y, z = tenth_resource["x"], tenth_resource["y"], tenth_resource["z"]

            # Calculate the magnitude of the resource vector
            magnitude = math.sqrt(x**2 + y**2 + z**2)
            if magnitude == 0:
                print("[Worker] Warning: 10th resource at origin, using default direction.")
                x, y, z = 0, 1, 0
                magnitude = 1.0

            # Normalize the resource vector to unit length
            unit_x = x / magnitude
            unit_y = y / magnitude
            unit_z = z / magnitude

            # Planet radius and buffer
            planet_radius = 100.0
            buffer = 5.0

            # Scale to the surface (radius) and add buffer
            target_position = [
                float(planet_position["x"] + (unit_x * (planet_radius + buffer))),
                float(planet_position["y"] + (unit_y * (planet_radius + buffer))),
                float(planet_position["z"] + (unit_z * (planet_radius + buffer)))
            ]
            print(f"[Worker] Target position (10th resource on surface with buffer): {target_position}")

            # Starting position
            start_position = [0, 100, 40]
            print(f"[Worker] Start position: {start_position}")

            # Number of cubes to spawn
            num_cubes = 10

            # Generate interpolated positions with buffer
            spawn_positions = interpolate_positions(start_position, target_position, num_cubes, buffer)
            print(f"[Worker] Generated {num_cubes} spawn positions with buffer:")
            for i, pos in enumerate(spawn_positions, 1):
                print(f"  {i}: {pos}")

            # Spawn cubes in separate threads
            for i, pos in enumerate(spawn_positions, 1):
                thread = threading.Thread(target=spawn_cube_thread, args=(i, pos, sockets))
                threads.append(thread)
                thread.start()
                time.sleep(0.5)  # Small delay between thread starts

            # Wait for all threads to complete spawning
            for thread in threads:
                thread.join()

            # Wait 5 seconds after last cube spawns
            print(f"[Worker] Waiting 5 seconds after last cube before forcing all connections closed...")
            time.sleep(5)

    except Exception as e:
        print("[Worker] Exception occurred:", e)
    finally:
        # Force close all sockets
        print("[Worker] Forcing closure of all TCP connections...")
        for sock in sockets:
            try:
                sock.shutdown(socket.SHUT_RDWR)
                print(f"[Worker] Socket {sock.fileno()} shutdown.")
            except (socket.error, OSError) as e:
                print(f"[Worker] Shutdown error for socket {sock.fileno()}: {e}")
            sock.close()
            print(f"[Worker] Socket {sock.fileno()} closed.")
        # Exit the process to ensure no lingering connections
        print("[Worker] Terminating process to close all ports...")
        #os._exit(0)

# Start the TCP operations in a separate thread
worker_thread = threading.Thread(target=tcp_worker)
worker_thread.start()
worker_thread.join(timeout=15.0)  # Give it 15 seconds to complete
if worker_thread.is_alive():
    print("Worker thread did not terminate, forcing process exit...")
    os._exit(0)
print("TCP worker thread has completed or was forced to exit.")

[Worker] Connected to server for planet data.
[Worker] Authentication response: auth_success
[Worker] Received planet data.
[Worker] Planet (0, 0, 0) center position: {'x': 0, 'y': 0, 'z': 0}
[Worker] Processing resources on planet (0, 0, 0) into a DataFrame...

Resources DataFrame (normalized):
              x         y         z
Index                              
1      0.373752  0.215376  0.252829
2     -0.038452  0.494592  0.062455
3      0.001948  0.479387 -0.142071
4      0.335547 -0.279743  0.243212
5     -0.032191  0.498757 -0.014325
6     -0.064523  0.455760 -0.195242
7      0.038518 -0.498494 -0.004483
8     -0.170413  0.027668  0.469248
9     -0.000631  0.499780  0.014816
10     0.331354  0.345534  0.144260
[Worker] Target position (10th resource on surface with buffer): [69.58438552780794, 72.56208670917846, 30.29450219308874]
[Worker] Start position: [0, 100, 40]
[Worker] Generated 10 spawn positions with buffer:
  1: [0.0, 100.0, 40.0]
  2: [7.749461244911638, 97.1753364

In [1]:
import numpy as np
import pandas as pd

def generate_trajectory_data(start_pos, end_pos, num_points=100):
    """Generate supervised training data with variable control gains near the target."""
    positions = []
    velocities = []
    forces = []

    # Initial state
    current_pos = np.array(start_pos, dtype=float)
    current_vel = np.array([0.0, 0.0, 0.0])
    dt = 0.1
    gravity = np.array([0, -9.81, 0])
    max_force = 1.5
    target = np.array(end_pos)
    
    for i in range(num_points):
        positions.append(current_pos.tolist())
        velocities.append(current_vel.tolist())
        
        error = target - current_pos
        distance = np.linalg.norm(error)
        
        # Use lower gains near the target for smoother deceleration.
        if distance < 20:  
            gain_x = 0.1
            gain_y = 0.05
            gain_z = -0.5
        else:
            gain_x = 0.3
            gain_y = 0.1
            gain_z = -2.0
        
        force = np.zeros(3)
        force[0] = np.clip(error[0] * gain_x, -max_force, max_force)
        # Y: Add a small offset for gravity compensation; adjust gain for deceleration.
        force[1] = np.clip(0.981 + error[1] * gain_y, -max_force, max_force)
        force[2] = np.clip(error[2] * gain_z, -max_force, 0.0)  # only allow non-positive force in Z
        
        # Cap force magnitude (if necessary)
        force_magnitude = np.linalg.norm(force)
        if force_magnitude > max_force:
            force = (force / force_magnitude) * max_force
        
        forces.append(force.tolist())
        
        # Update physics
        acceleration = force + gravity
        current_vel += acceleration * dt
        current_pos += current_vel * dt
    
    # Ensure last state is exactly the target
    positions[-1] = end_pos
    velocities[-1] = [0, 0, 0]
    forces[-1] = [0, 0, 0]

    return positions, velocities, forces

# Define start and end positions
start_pos = [0, 100, 40]
end_pos = [69.58438552780794, 72.56208670917846, 30.29450219308874]

# Generate data
positions, velocities, forces = generate_trajectory_data(start_pos, end_pos)

# Create DataFrame
data = []
for i in range(len(positions)):
    data.append({
        'pos_x': positions[i][0],
        'pos_y': positions[i][1],
        'pos_z': positions[i][2],
        'vel_x': velocities[i][0],
        'vel_y': velocities[i][1],
        'vel_z': velocities[i][2],
        'force_x': forces[i][0],
        'force_y': forces[i][1],
        'force_z': forces[i][2]
    })

df = pd.DataFrame(data)
print("Properly Fixed Supervised Training Data:")
print(df.head(10))
print(df.tail(10))

# Save to CSV
df.to_csv('cube_trajectory_data_properly_fixed.csv', index=False)
print("Data saved to 'cube_trajectory_data_properly_fixed.csv'")


Properly Fixed Supervised Training Data:
      pos_x       pos_y  pos_z     vel_x     vel_y  vel_z   force_x   force_y  \
0  0.000000  100.000000   40.0  0.000000  0.000000    0.0  1.060660 -1.060660   
1  0.010607   99.891293   40.0  0.106066 -1.087066    0.0  1.060660 -1.060660   
2  0.031820   99.673880   40.0  0.212132 -2.174132    0.0  1.060660 -1.060660   
3  0.063640   99.347760   40.0  0.318198 -3.261198    0.0  1.060660 -1.060660   
4  0.106066   98.912934   40.0  0.424264 -4.348264    0.0  1.060660 -1.060660   
5  0.159099   98.369401   40.0  0.530330 -5.435330    0.0  1.060660 -1.060660   
6  0.222739   97.717161   40.0  0.636396 -6.522396    0.0  1.060660 -1.060660   
7  0.296985   96.956215   40.0  0.742462 -7.609462    0.0  1.075464 -1.045647   
8  0.381986   96.086712   40.0  0.850009 -8.695027    0.0  1.107031 -1.012167   
9  0.478057   95.108988   40.0  0.960712 -9.777244    0.0  1.143402 -0.970893   

   force_z  
0      0.0  
1      0.0  
2      0.0  
3      0.0  
4 

In [2]:
from sklearn.neural_network import MLPRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np
import joblib

# Load fixed data
df = pd.read_csv('cube_trajectory_data_properly_fixed.csv')

# Features and target
X = df[['pos_x', 'pos_y', 'pos_z', 'vel_x', 'vel_y', 'vel_z']].values
y = df[['force_x', 'force_y', 'force_z']].values

# Split into training and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Scale the features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Train the MLP regressor
model = MLPRegressor(hidden_layer_sizes=(200, 150, 100), max_iter=10000, random_state=42, learning_rate_init=0.0005)
model.fit(X_train_scaled, y_train)

# Evaluate the model
score = model.score(X_test_scaled, y_test)
print(f"Model R^2 Score: {score}")

# Save the model and scaler
joblib.dump(model, 'cube_force_model_properly_fixed.pkl')
joblib.dump(scaler, 'cube_force_scaler_properly_fixed.pkl')
print("Model and scaler saved as 'cube_force_model_properly_fixed.pkl' and 'cube_force_scaler_properly_fixed.pkl'")


Model R^2 Score: 0.4899174177796371
Model and scaler saved as 'cube_force_model_properly_fixed.pkl' and 'cube_force_scaler_properly_fixed.pkl'


In [ ]:
import socket
import json
import time
import threading
import math
import os
import numpy as np

# Server configuration
SERVER_IP = '192.168.0.116'
SERVER_PORT = 14000
SHARED_PASSWORD = "my_secure_password"
END_OF_MSG = "<???DONE???---"

socket.setdefaulttimeout(1.0)

def send_message(sock, message):
    full_message = message + END_OF_MSG
    try:
        sock.sendall(full_message.encode('utf-8'))
    except (socket.error, OSError) as e:
        print(f"[Thread] Send error: {e}")

def receive_message(sock):
    buffer = ""
    try:
        while END_OF_MSG not in buffer:
            data = sock.recv(1024).decode('utf-8')
            if not data:
                break
            buffer += data
    except (socket.timeout, socket.error, OSError):
        pass
    return buffer.replace(END_OF_MSG, "").strip()

def spawn_and_control_cube(cube_id, start_position, target_position):
    sock = None
    try:
        sock = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        sock.connect((SERVER_IP, SERVER_PORT))
        print(f"[Cube {cube_id}] Connected to server.")

        # Authenticate
        send_message(sock, SHARED_PASSWORD)
        auth_response = receive_message(sock)
        print(f"[Cube {cube_id}] Authentication response:", auth_response)
        if auth_response != "auth_success":
            raise Exception(f"Cube {cube_id} authentication failed.")

        # Spawn the cube
        spawn_cube_cmd = {
            "type": "spawn_cube",
            "position": start_position,
            "rotation": [0, 0, 0]
        }
        send_message(sock, json.dumps(spawn_cube_cmd))
        print(f"[Cube {cube_id}] Spawned at {start_position}")

        # Control parameters for a gentle, proportional controller.
        k = 0.05  # low proportional gain for gentle force
        max_force_magnitude = 1.0  # cap the force to 1.0 (or lower as needed)
        target = np.array(target_position)
        tolerance = 5.0

        # Control loop: run for up to 10 seconds
        start_time = time.time()
        while time.time() - start_time < 10:
            # Get cube state
            send_message(sock, json.dumps({"type": "get_cube_state"}))
            state_response = receive_message(sock)
            if not state_response:
                continue
            try:
                state = json.loads(state_response)
            except Exception as e:
                print(f"[Cube {cube_id}] JSON decode error: {e}")
                continue

            # Extract current position and velocity
            pos = np.array(state.get('position', [0, 0, 0]))
            vel = np.array(state.get('linear_velocity', [0, 0, 0]))

            # Compute error and determine force using a low gain
            error = target - pos
            distance_to_target = np.linalg.norm(error)
            if distance_to_target < tolerance:
                print(f"[Cube {cube_id}] Near target, stopping. Position: {pos.tolist()}")
                break

            # Proportional control: compute a low force toward the target
            force_total = k * error

            # Cap the force magnitude if needed
            force_magnitude = np.linalg.norm(force_total)
            if force_magnitude > max_force_magnitude:
                force_total = (force_total / force_magnitude) * max_force_magnitude

            # Build and send the command to apply force
            apply_force_cmd = {
                "type": "apply_force",
                "force": force_total.tolist()
            }
            send_message(sock, json.dumps(apply_force_cmd))
            print(f"[Cube {cube_id}] Pos: {pos.tolist()}, Vel: {vel.tolist()}, Force: {force_total.tolist()}")

            time.sleep(0.1)

    except Exception as e:
        print(f"[Cube {cube_id}] Exception occurred:", e)
    finally:
        if sock:
            try:
                sock.shutdown(socket.SHUT_RDWR)
            except:
                pass
            sock.close()
            print(f"[Cube {cube_id}] Connection closed.")
        os._exit(0)

# Start and target positions
start_position = [0, 100, 40]
target_position = [69.58438552780794, 72.56208670917846, 30.29450219308874]

# Spawn and control the cube
worker_thread = threading.Thread(target=spawn_and_control_cube, args=(1, start_position, target_position))
worker_thread.start()
worker_thread.join(timeout=15.0)
if worker_thread.is_alive():
    print("Worker thread did not terminate, forcing exit...")
    os._exit(0)
print("Cube control completed or forced to exit.")


[Cube 1] Connected to server.
[Cube 1] Authentication response: auth_success
[Cube 1] Spawned at [0, 100, 40]
[Cube 1] Position: [0.0, 99.997475, 39.99899], Velocity: [0.0, -0.1516512, -0.060660485], Applied Force: [ 0.97188446 -1.14226954  0.0257079 ]
[Cube 1] Position: [0.08057816, 99.834755, 39.97391], Velocity: [0.96356773, -2.1890056, -0.3971164], Applied Force: [ 1.11943213 -0.99840976  0.00704627]
[Cube 1] Position: [0.30344334, 99.41129, 39.900337], Velocity: [2.05888, -4.208039, -0.80875176], Applied Force: [ 1.25642083 -0.68462996 -0.04869042]
[Cube 1] Position: [0.61211026, 98.88327, 39.79465], Velocity: [3.2804918, -5.750632, -1.2128345], Applied Force: [ 1.3028971  -0.2618251  -0.07963832]
[Cube 1] Position: [1.0458965, 98.23695, 39.646168], Velocity: [4.532696, -6.8582015, -1.6444448], Applied Force: [ 1.3261028   0.11616106 -0.1064263 ]
[Cube 1] Position: [1.6058437, 97.511894, 39.452507], Velocity: [5.790901, -7.579509, -2.0991607], Applied Force: [ 1.33059801  0.343990

In [ ]:
# Close the connection to the server gracefully.
sock.close()
print("Disconnected from server.")
